In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)
pd.set_option('display.max_colwidth', 199)

from datetime import datetime
import re
import os
import os.path
import math
import sys
import csv
import glob
import seaborn as sns
import numpy as np
import shap 
from collections import Counter

# import mlflow
# import mlflow.sklearn
# import pysftp

from sklearn.metrics import confusion_matrix
import seaborn as sns
pd.options.display.float_format = "{:.2f}".format

import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import traceback
import string

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm 

import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, accuracy_score, log_loss, roc_auc_score, f1_score

%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/data_preparation/utils.py"
%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/visualization/common_visualization.py"
%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/data_preparation/common_eda.py"
%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/modelling/regression_model.py"
%run "/Users/chau/minhchau/jupyter_projects/jupyter_project_template-master/src/data_preparation/common_dictionary.py"

print("current working folder: ", os.getcwd())
user_name = "chaupham"
# print("notebook kernel id: ", get_notebook_kernel_id())

Using TensorFlow backend.


current working folder:  /Users/chau/minhchau/jupyter_projects/plant_project


In [2]:
df = pd.read_csv("./data/transcriptome_cpm_all_expressed_genes.csv")
df.shape                 

(20450, 25)

In [5]:
df.head()

,atID,wthp6,wthp6_2,wtlp6,wtlp6_2,wthp5,wthp5_2,wtlp5,wtlp5_2,wtal,wtal_2,wtfe,wtfe_2,stop1hp6,stop1hp6_2,stop1lp6,stop1lp6_2,stop1hp5,stop1hp5_2,stop1lp5,stop1lp5_2,stop1al,stop1al_2,s1fe,s1fe_2
0,AT1G01020,18.51,17.60,17.80,20.36,18.99,21.93,18.29,21.48,19.01,17.58,18.21,18.19,19.27,17.38,20.43,22.19,21.40,22.06,18.24,21.52,21.57,17.08,16.60,21.07
1,AT1G01030,0.82,1.07,0.68,0.45,0.54,0.40,0.56,0.29,0.22,0.81,0.93,0.62,0.66,0.67,1.10,0.41,1.09,0.74,1.05,0.51,0.60,0.53,1.13,1.34
2,AT1G01040,36.57,34.09,31.92,53.51,37.06,34.78,43.44,49.22,42.09,36.11,32.94,17.10,33.04,27.38,35.39,27.64,31.81,40.87,38.15,37.58,43.88,38.97,23.65,27.46
3,AT1G01050,103.58,109.89,103.21,96.22,100.88,106.00,97.60,100.95,89.76,92.77,91.97,112.10,99.01,113.33,109.44,120.67,98.69,97.40,105.70,111.05,86.29,85.37,104.54,106.91
4,AT1G01060,1.94,1.78,2.93,2.88,3.15,3.67,2.44,2.13,1.34,1.89,3.52,1.89,2.09,3.13,2.51,2.11,1.54,2.36,1.15,3.60,1.33,1.12,5.44,3.36


In [3]:
df.columns

Index(['atID', 'wthp6', 'wthp6_2', 'wtlp6', 'wtlp6_2', 'wthp5', 'wthp5_2', 'wtlp5', 'wtlp5_2',
       'wtal', 'wtal_2', 'wtfe', 'wtfe_2', 'stop1hp6', 'stop1hp6_2', 'stop1lp6', 'stop1lp6_2',
       'stop1hp5', 'stop1hp5_2', 'stop1lp5', 'stop1lp5_2', 'stop1al', 'stop1al_2', 's1fe',
       's1fe_2'],
      dtype='object')

In [34]:
all_cols = ['wthp6',  'wtlp6', 'wthp5',  'wtlp5',  'wtal',  'wtfe', 
            'stop1hp6',  'stop1lp6', 'stop1hp5', 'stop1lp5',  'stop1al',  's1fe']

data = df[["atID"]]
for c in all_cols:
    data[c] = (df[c]  + df[f"{c}_2"])/2
    
data.columns = ['atID', 'wthp6', 'wtlp6', 'wthp5', 'wtlp5', 'wtal', 'wtfe', 
                's1hp6', 's1lp6', 's1hp5', 's1lp5', 's1al', 's1fe']


In [35]:
data.columns

Index(['atID', 'wthp6', 'wtlp6', 'wthp5', 'wtlp5', 'wtal', 'wtfe', 's1hp6', 's1lp6', 's1hp5',
       's1lp5', 's1al', 's1fe'],
      dtype='object')

In [36]:
# data.to_csv("./data/data_ALL_raw.csv", index=False)

In [37]:
new_cols = ['wthp6', 'wtlp6', 'wthp5', 'wtlp5', 'wtal', 'wtfe', 
                's1hp6', 's1lp6', 's1hp5', 's1lp5', 's1al', 's1fe']
def norm_data(x):
    max_x = np.amax(x[new_cols])
    x = x[new_cols]/max_x
    return x

In [38]:
data_norm = data[["atID"]]

In [39]:
data_norm[new_cols] = data.apply(norm_data, 1)
data_norm

,atID,wthp6,wtlp6,wthp5,wtlp5,wtal,wtfe,s1hp6,s1lp6,s1hp5,s1lp5,s1al,s1fe
0,AT1G01020,0.83,0.88,0.94,0.92,0.84,0.84,0.84,0.98,1.00,0.91,0.89,0.87
1,AT1G01030,0.76,0.46,0.38,0.35,0.42,0.62,0.54,0.61,0.74,0.63,0.46,1.00
2,AT1G01040,0.76,0.92,0.78,1.00,0.84,0.54,0.65,0.68,0.78,0.82,0.89,0.55
3,AT1G01050,0.93,0.87,0.90,0.86,0.79,0.89,0.92,1.00,0.85,0.94,0.75,0.92
4,AT1G01060,0.42,0.66,0.77,0.52,0.37,0.61,0.59,0.52,0.44,0.54,0.28,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20445,ATMG01130,0.59,1.00,0.31,0.52,0.30,0.34,0.13,0.16,0.30,0.19,0.24,0.17
20446,ATMG01200,1.00,0.91,0.26,0.17,0.25,0.44,0.35,0.33,0.26,0.22,0.50,0.14
20447,ATMG01360,0.40,0.66,0.41,0.33,0.86,0.71,0.27,1.00,0.54,0.86,0.82,0.93
20448,ATMG01370,0.88,1.00,0.53,0.18,0.51,0.45,0.27,0.38,0.13,0.24,0.33,0.25


In [40]:
# data_norm.to_csv("./data/data_ALL_norm.csv", index=False)

In [41]:
data_norm.to_csv("/Users/chau/minhchau/git_/C/plant_genetics_visualization/data/data_ALL_norm.csv", index=False)
data.to_csv("/Users/chau/minhchau/git_/C/plant_genetics_visualization/data/data_ALL_raw.csv", index=False)